<a href="https://colab.research.google.com/github/emmanuellfc/EM_Tufts_Fall2023/blob/main/GeneralFiniteDifferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Package dependencies.
!apt install libcgal-dev libeigen3-dev
!pip install -U pygalmesh
!pip install networkx

In [ ]:
# Imports.
import pygalmesh
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Represents a mesh (contains a graph) for the system we're trying to solve.
class FiniteDifferences:
  # Initialize the object with default values.
  def __init__(self):
    self.edge_points = None # List of (x,y) points that form the boundary of the 2D space.
    self.region_points = None # List of (x,y) points that sample the region defined by the boundary.
    self.graph = nx.Graph() # Graph data structure representing the region.
    self.boundary_conditions = [] # Not sure what the elements of this will look like yet.
    self.systems_matrix = None # m x m matrix where m is the number of points in the space.

  # Make a list of points from some edge conditions. This makes a surface "mesh" from a boundary definition. The boundary is a list of points.
  def make_points_from_boundary(self):
    pass
  # Make a mesh from a list of points. Don't make neighbors out of points that are too far away.
  def make_mesh_from_points(self):
    pass
  # Only callable once a graph exists. Make the C matrix element by element.
  def make_systems_matrix(self):
    pass
  # Inform the method of boundary conditions that exist for the problem.
  def inform_with_boundary_condition(self):
    pass
  # Only callable once a C matrix exists. Try to solve the C matrix.
  def solve_systems_matrix(self):
    pass
  # Profit???